In [37]:
import boto3
from botocore.config import Config
import pandas as pd

SOURCE_BUCKET = "deafrica-sentinel-1-staging-frankfurt"

In [49]:
paths = set(pd.read_csv("missing-data.txt").path)

In [51]:
for path in paths:
    print(path)
    break

s1_rtc/S09E011/2020/05/24/03C9B1/


In [31]:
my_config = Config(region_name='eu-central-1')
s3 = boto3.client("s3", config=my_config)
s3r = boto3.resource('s3', config=my_config)
bucket = s3r.Bucket(SOURCE_BUCKET)

In [24]:
resp = s3.list_object_versions(Bucket=SOURCE_BUCKET, Prefix=path)
del_markers = {item['Key']: item['VersionId'] for item in resp['DeleteMarkers'] if item['IsLatest'] == True}

{'s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_ANGLE.tif': 'vEPZzJwqVtycKlFj8eB2fv8fKYQnwypk',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_AREA.tif': 'vEPZzJwqVtycKlFj8eB2fv8fKYQnwypk',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_MASK.tif': 'vEPZzJwqVtycKlFj8eB2fv8fKYQnwypk',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_VH.tif': 'vEPZzJwqVtycKlFj8eB2fv8fKYQnwypk',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_VV.tif': 'vEPZzJwqVtycKlFj8eB2fv8fKYQnwypk',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_metadata.json': 'EDle7fCRThw5KyFoTgjOd7EK0ib4A0Sa',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_metadata.xml': 'EDle7fCRThw5KyFoTgjOd7EK0ib4A0Sa',
 's1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_userdata.json': 'vEPZzJwqVtycKlFj8eB2fv8fKYQnwypk'}

In [60]:
for key, version in del_markers.items():
    head = s3.head_object(Bucket=SOURCE_BUCKET, Key=key)

    replication_status = head.get("ReplicationStatus")
    if replication_status and replication_status != "COMPLETED":
        print(f"Replicating {key}")
        object = bucket.Object(key)
        object.delete(VersionId=version)
        s3r.Object(SOURCE_BUCKET, key).copy_from(
            CopySource=f'{SOURCE_BUCKET}/{key}',
            MetadataDirective='REPLACE'
        )
    else:
        print(f"Key is replicated: {key}")

Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_ANGLE.tif
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_AREA.tif
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_MASK.tif
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_VH.tif
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_VV.tif
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_metadata.json
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_metadata.xml
Key is replicated: s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_userdata.json


In [35]:
s3r.Object(SOURCE_BUCKET,'s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_userdata.json').copy_from(CopySource=f'{SOURCE_BUCKET}/s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_userdata.json', MetadataDirective='REPLACE')


{'ResponseMetadata': {'RequestId': '0VEK9FMVDFM39407',
  'HostId': '8+V+aD+kLFgX+AiovyEpFOYhJzFMcakOhdJ3KCG3O/FWTKdxVapWyQGD7hDWIL3V3MopYW/jB5k=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8+V+aD+kLFgX+AiovyEpFOYhJzFMcakOhdJ3KCG3O/FWTKdxVapWyQGD7hDWIL3V3MopYW/jB5k=',
   'x-amz-request-id': '0VEK9FMVDFM39407',
   'date': 'Thu, 06 May 2021 04:35:53 GMT',
   'x-amz-copy-source-version-id': 'kyA2FiLZfx.7Yaw_hDaXmbCP2SY_SUjw',
   'x-amz-version-id': 'hFPyPsQHyjLmAXr76gwzOWjJIWP8b4Nr',
   'x-amz-expiration': 'expiry-date="Fri, 14 May 2021 00:00:00 GMT", rule-id="Delete marker at 7 days and delete object at 14 days MANUAL"',
   'content-type': 'application/xml',
   'content-length': '234',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Expiration': 'expiry-date="Fri, 14 May 2021 00:00:00 GMT", rule-id="Delete marker at 7 days and delete object at 14 days MANUAL"',
 'CopySourceVersionId': 'kyA2FiLZfx.7Yaw_hDaXmbCP2SY_SUjw',
 'VersionId': 'hFPyPsQHyjLmAXr76gwzOWjJIWP8b4Nr',


In [52]:
o = bucket.Object(key)

In [55]:
o

s3.Object(bucket_name='deafrica-sentinel-1-staging-frankfurt', key='s1_rtc/N31E004/2020/01/23/025B80/s1_rtc_025B80_N31E004_2020_01_23_userdata.json')

In [58]:
head = s3.head_object(Bucket=SOURCE_BUCKET, Key=key)

In [59]:
head

{'ResponseMetadata': {'RequestId': '4AS0EEM52JG7T8DM',
  'HostId': 'O8JWSTbOhHL46FANJftzsbYE3/lO4p/TL1A0iojYwJLnIlQvDEV1aDHtOGQQXir5TDflvC0lHrA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'O8JWSTbOhHL46FANJftzsbYE3/lO4p/TL1A0iojYwJLnIlQvDEV1aDHtOGQQXir5TDflvC0lHrA=',
   'x-amz-request-id': '4AS0EEM52JG7T8DM',
   'date': 'Thu, 06 May 2021 05:00:38 GMT',
   'x-amz-replication-status': 'COMPLETED',
   'last-modified': 'Thu, 06 May 2021 04:35:53 GMT',
   'x-amz-expiration': 'expiry-date="Fri, 14 May 2021 00:00:00 GMT", rule-id="Delete marker at 7 days and delete object at 14 days MANUAL"',
   'etag': '"b8a447770b6361b99b845028d27403d2"',
   'x-amz-tagging-count': '1',
   'x-amz-version-id': 'hFPyPsQHyjLmAXr76gwzOWjJIWP8b4Nr',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'content-length': '2474',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'Expiration': 'expiry-date="Fri, 14 May 2021 00:00:00 GMT", rule-id="Delete 